### Gruppo 5.a: Castagnacci Giulia (581749), Giordano Elisabetta (536265)

### Analisi linefragments

In [1]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads

### Dati in input

In [2]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);                                          #vertici del modello 2D
V3 = hcat([[0.,0,0],[1,0,3],[1,1,2],[0,1,1],[2,1,0]]...);                               #vertici del modello 3D
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];                                                   #spigoli del modello
bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];  #bounding box
dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])  #dizionario intervallo/indice
cov = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]    #intersezioni tra bounding box

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

### Funzioni di supporto

In [3]:
function spaceindex(model::Lar.LAR)::Array{Array{Int,1},1}
	V,CV = model[1:2]
	dim = size(V,1)
	cellpoints = [ V[:,CV[k]]::Lar.Points for k=1:length(CV) ]
	#----------------------------------------------------------
	bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
	xboxdict = Lar.coordintervals(1,bboxes)
	yboxdict = Lar.coordintervals(2,bboxes)
	# xs,ys are IntervalTree type
	xs = IntervalTrees.IntervalMap{Float64, Array}()
	for (key,boxset) in xboxdict
		xs[tuple(key...)] = boxset
	end
	ys = IntervalTrees.IntervalMap{Float64, Array}()
	for (key,boxset) in yboxdict
		ys[tuple(key...)] = boxset
	end
	xcovers = Lar.boxcovering(bboxes, 1, xs)
	ycovers = Lar.boxcovering(bboxes, 2, ys)
	covers = [intersect(pair...) for pair in zip(xcovers,ycovers)]

	if dim == 3
		zboxdict = Lar.coordintervals(3,bboxes)
		zs = IntervalTrees.IntervalMap{Float64, Array}()
		for (key,boxset) in zboxdict
			zs[tuple(key...)] = boxset
		end
		zcovers = Lar.boxcovering(bboxes, 3, zs)
		covers = [intersect(pair...) for pair in zip(zcovers,covers)]
	end
	# remove each cell from its cover
	for k=1:length(covers)
		covers[k] = setdiff(covers[k],[k])
	end
	return covers
end

function intersection(line1,line2)
	x1,y1,x2,y2 = vcat(line1...)
	x3,y3,x4,y4 = vcat(line2...)

	# intersect lines e1,e2
	det = (x4-x3)*(y1-y2)-(x1-x2)*(y4-y3)
	if det != 0.0
		a = 1/det
		b = [y1-y2 x2-x1; y3-y4 x4-x3]
		c = [x1-x3; y1-y3]
		(β,α) = a * b * c
	else
		if (y1==y2) == (y3==y4) || (x1==x2) == (x3==x4) 	# segments collinear
			 return nothing
		else
			 # segments parallel: no intersection
			 return nothing
		end
	end
	return α,β
end

intersection (generic function with 1 method)

### Versione iniziale di linefragments
Funzione che ordina i segmenti intersecati di ogni segmento. Per ogni segmento, se interseca con qualcosa, prende i punti (x1, y1), (x2, y2) del segmento i-eisimo, lo confronta con tutti gli altri segmenti presenti nel suo indice spaziale “Sigma” che fornisce il sottoinsieme di linee i cui box di contenimento intersecano il box di ciascuna linea di input (dato da EV), e restituisce i parametri (alfa e beta) necessari a determinare il punto di intersezione tra coppie di segmenti. Se le rette si intersecano, si verifica che i parametri alfa e beta siano ammissibili, se lo sono vengono immagazzinati nella struttura dati “params”.


In [4]:
function linefragments(V,EV,Sigma)
	# remove the double intersections by ordering Sigma
	m = length(Sigma)
	sigma = map(sort,Sigma)
	reducedsigma = sigma
	# pairwise parametric intersection
	params = Array{Float64,1}[[] for i=1:m]
	for h=1:m
		if sigma[h] ≠ []
			line1 = V[:,EV[h]]
			for k in sigma[h]
				line2 = V[:,EV[k]]
				out = Lar.intersection(line1,line2)
				if out ≠ nothing
					α,β = out
					if 0<=α<=1 && 0<=β<=1
						push!(params[h], α)
						push!(params[k], β)
					end
				end
			end
		end
	end
	# finalize parameters of fragmented lines
	fragparams = []
	for line in params
		push!(line, 0.0, 1.0)
		line = sort(collect(Set(line)))
		push!(fragparams, line)
	end
	return fragparams
end

linefragments (generic function with 1 method)

### Analisi del comportamento e dei tempi della versione iniziale

In [5]:
Sigma = spaceindex((V,EV))
@btime linefragments(V,EV,Sigma)

  41.300 μs (800 allocations: 25.34 KiB)


5-element Vector{Any}:
 [0.0, 1.0]
 [0.0, 0.5, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 0.5, 1.0]

In [6]:
@code_warntype linefragments(V,EV,Sigma)

MethodInstance for 

linefragments(::

Matrix{Float64}, ::Vector{Vector{Int64}}, ::Vector{Vector{Int64}})
  from linefragments(V, EV, Sigma) in Main at c:\Users\giord\eclipse-SIW\LARSplitting2D\notebooks\refactoringAnalisi_linefragments.ipynb:1
Arguments
  #self#

::Core.Const(linefragments)
  V::Matrix{Float64}
  EV::Vector{Vector{Int64}}
  Sigma::Vector{Vector{Int64}}
Locals
  @_5::Union{Nothing, Tuple{Vector{Float64}, Int64}}
  @_6::Union{Nothing, Tuple{Int64, Int64}}
  fragparams::Vector{Any}
  params::Vector{Vector{Float64}}
  reducedsigma::Vector{Vector{Int64}}
  sigma::Vector{Vector{Int64}}
  m::Int64
  @_12::Union{Nothing, Tuple{Int64, Int64}}
  @_13::Int64
  i::Int64
  @_15::Union{Nothing, Tuple{Int64, Int64}}
  h::Int64
  line1::Matrix{Float64}
  @_18::Any
  k::Int64
  β::Any
  α::Any
  out::Any
  line2::Matrix{Float64}
  line::Vector{Float64}
  @_25::Any
  @_26::Any
Body::Vector{Any}


1 ──

...

false)
13 ┄        goto #19 if not @_25
14 ─ %81  = (0 <= β)::Any
└───        goto #16 if not %81
15 ─        (@_26 = β <= 1)
└───        goto #17
16 ─        (@_26 = false)
17 ┄        goto #19 if not @_26
18 ─ %87  = Base.getindex(params, h)::Vector{Float64}
│           Main.push!(%87, α)
│    %89  = Base.getindex(params, k)::Vector{Float64}
└───        Main.push!(%89, β)


19 ┄        (@_15 = Base.iterate(%52, %62))
│    %92  = (@_15 === nothing)::Bool
│    %93  = Base.not_int(%92)::Bool
└───        goto #21 if not %93
20 ─        goto #9
21 ┄        (@_6 = Base.iterate(%36, %45))
│    %97  = (@_6 === 

nothing)::Bool
│    %98  = Base.not_int(%97)::Bool
└───        goto #23 if not %98
22 ─        goto #7
23 ┄        (fragparams = Base.vect())
│    %102 = params::Vector{Vector{Float64}}
│           (@_5 = Base.iterate(%102))
│    %104 = (@_5 === nothing)::Bool
│    %105 = Base.not_int(%104)::Bool
└───        goto #26 if not %105
24 ┄ %107 = @_5::Tuple{Vector{Float64}, Int64}
│           (line = Core.getfield(%107, 1))
│    %109 = Core.getfield(%107, 2)::Int64
│           Main.push!(line

, 0.0, 1.0)
│    %111 = Main.Set(line)::Set{Float64}
│    %112 = Main.collect(%111)::Vector{Float64}
│           (line = Main.sort(%112))
│           Main.push!(fragparams, line)
│           (@_5 = Base.iterate(%102, %109))
│    %116 = (@_5 === nothing)::Bool
│    %117 = Base.not_int(%

116)::Bool
└───        goto #26 if not %117
25 ─        goto #24
26 ┄        return fragparams



In [7]:
@benchmark linefragments(V,EV,Sigma)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  43.600 μs …  12.599 ms  ┊ GC (min … max): 0.00% … 98.66%
 Time  (median):     46.300 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   60.058 μs ± 201.040 μs  ┊ GC (mean ± σ):  7.23% ±  2.21%

  ▇█▆▄▄▂▃▃▃▃▂▂▁    ▁  ▁  ▂▅▃▁            ▁▁                    ▂
  ███████████████▇██▇█████████▇▆▆▆█████▇████▇▇▇▆▅▆▆▇▇▇▇██▇▇▇▇▇ █
  43.6 μs       Histogram: log(frequency) by time       112 μs <

 Memory estimate: 25.34 KiB, allocs estimate: 800.

### Versione parallelizzata di linefragments 
abbiamo paralellizzato la funzione attraverso la  macro @threads,  notando un lieve miglioramento delle performance. 

In [8]:
function linefragments2(V,EV,sigma)
    m = length(sigma) 
    sigma = map(sort,sigma) 
    params = Array{Array{Float64,1}}(undef,m)
    for i=1:m
        params[i] = []
    end
    line1=[0.0 0.0; 0.0 0.0]
    line2=[0.0 0.0; 0.0 0.0]
    @threads for h=1:m
        if sigma[h] ≠ []
            line1 = V[:,EV[h]]
            @threads for k in sigma[h]
            line2 = V[:,EV[k]]
                out = intersection(line1,line2) 
                if out ≠ ()
                    if 0<=out[1]<=1 && 0<=out[2]<=1
                        push!(params[h], out[1])
                        push!(params[k], out[2])
                    end
                end
            end
        end
        end
    len = length(params)
    @threads for i=1:len
        push!(params[i], 0.0, 1.0)
        params[i] = sort(collect(Set(params[i])))
    end
    return params
end

linefragments2 (generic function with 1 method)

### Analisi del comportamento e dei tempi della versione modificata

In [9]:
@btime linefragments2(V,EV,Sigma)

  73.900 μs (816 allocations: 26.62 KiB)


5-element Vector{Vector{Float64}}:
 [0.0, 1.0]
 [0.0, 0.5, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 0.5, 1.0]

In [10]:
@benchmark linefragments2(V,EV,Sigma)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   74.100 μs …  18.328 ms  ┊ GC (min … max): 0.00% … 98.00%
 Time  (median):     120.200 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   121.672 μs ± 280.463 μs  ┊ GC (mean ± σ):  4.84% ±  2.20%

  █▆▄▂▄▄▃▁▂▁▄▃▂▁ ▃▁ ▇▆▃▃▃▃▄▄▄▃▂ ▅▅▃▁             ▁▂  ▃▁         ▂
  █████████████████▇█████████████████▇▆▆▅▅▄▆▆▇▇▇███▇████▇▇▇▅▅▄▅ █
  74.1 μs       Histogram: log(frequency) by time        226 μs <

 Memory estimate: 26.62 KiB, allocs estimate: 816.

## Test

In [11]:
using Test

@testset "linefragments Tests" begin
    V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);
    EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];
    @test Lar.spaceindex((V,EV)) == 
    [[4, 5, 2], [1, 3, 5], [4, 5, 2], [1, 3, 5], [4, 1, 3, 2]]
    Sigma = [[4, 5, 2], [1, 3, 5], [4, 5, 2], [1, 3, 5], [4, 1, 3, 2]]
    @test Lar.linefragments(V,EV,Sigma) ==
    [[0.0, 1.0], [0.0, 0.5, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 0.5, 1.0]]
end

Test Summary:       | Pass  Total
linefragments Tests |    2  

    2


Test.DefaultTestSet("linefragments Tests", Any[], 2, false, false)

![](https://github.com/GiuliaCastagnacci/LARSplitting2D/blob/main/docs/plot/screenTest/test_linefragments.png?raw=true)